In [3]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
jokes_info = dataiku.Dataset("jokes_tokenized_by_text_joined")
jokes_info_df = jokes_info.get_dataframe()
my_jokes = dataiku.Dataset("jokes_copy_to_join_prepared")
my_jokes_df = my_jokes.get_dataframe()
test = dataiku.Dataset("test")
test_df = test.get_dataframe()
rating_df=test_df.copy()

# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.



In [4]:
jokes_info_df.head()

dfm  count mot  user  moyenne_user  moyenneBlague
0    1      3  00     0          6.26       1.687813
1    1      3  00     4          8.11       1.687813
2    1      3  00     8         -1.60       1.687813
3    1      3  00    11         -0.29       1.687813
4    1      3  00    16         -2.28       1.687813

In [5]:
my_jokes_df.head()

id                                               text
0  95  ["just","thought","critic","someon","walk","mi...
1   1  ["man","visit","doctor","doctor","say","bad","...
2  83  ["woman","say","place","mess","c","mon","need"...
3  17  ["mani","men","take","screw","light","bulb","o...
4  40  ["mani","irishmen","take","chang","lightbulb",...

In [6]:
rating_df.head()

user  blague  rating
0     0      20    1.17
1     0       7   -0.78
2     0      16   -0.19
3     0      44   -5.87
4     0      18    2.96

In [7]:
import json
import math
n=len(set(rating_df["blague"]))  # N pour la formule
myprediction=[]
for i,j in rating_df.iterrows():
    #if(j[2]==99):
    #print(j[0])
    #blague=my_jokes_df[ ( my_jokes_df["user"]==j[0] ) & ( my_jokes_df["blague"]==int(j[1]) ) ].text
    tmpline=my_jokes_df[ ( my_jokes_df["id"]==j[1])      ]
    for k,l in tmpline.iterrows():
        blague=json.loads(l[1])

    #print("blah",len(blague))
    #notesUtils=jokes_info_df[jokes_info_df.user==j[0]]#note donné par le meme utilisateur

    predict=0
    i=0
    for mot in blague:
        dd=jokes_info_df[jokes_info_df["mot"]==mot]["dfm"].values
        if(len(dd)==0):
            continue

        #print(dd)
        dfmmot=dd[0]
        #print(mot+str(dfmmot))


        note=jokes_info_df[
            ( jokes_info_df["user"] == j[0] )
            &
            ( jokes_info_df["mot"]==mot   )
        ].moyenne_user.values

        if(len(note)>0):
            #print(note[0])
            #predict +=   math.log(n/dfmmot)  *  note[0]# on normalise avec +1 car sinon risquons d'avoir un 0
            predict +=   note[0]
            i+=1
        #print("note",note)
    #print("izan",predict)
    if(i!=0):
        note_moy=predict/i
    else:
        note_moy=0
    #print("je vais mettre a jour",note_moy)

    myprediction.append(note_moy)
    #print("note initiale",j[2],"prediction",note_moy)
rating_df["predictionNote"]=myprediction

In [8]:
moteur_recomandation1_df = rating_df # Compute a Pandas dataframe to write into moteur_recomandation1


# Write recipe outputs
moteur_recomandation1 = dataiku.Dataset("moteur_recomandation1")
moteur_recomandation1.write_with_schema(moteur_recomandation1_df)

1207 rows successfully written (ptnvxQ6nxR)
